In [1]:
import pandas as pd

<H1>Load Dataset

In [2]:
#Baca Hasil Scraping
df = pd.read_csv('shopee_reviews.csv')
df.dropna(subset=['content'], inplace=True)

print('Jumlah data', len(df))

Jumlah data 20000


In [3]:
df.head()

,userName,content,score,at
0,Gerald Yoseka,bagus,5,2025-04-08 13:21:45
1,Yuliana Ziraluo,bagus sekali untuk mencari barang yang dibutuhkan,5,2025-04-08 13:21:38
2,Wahyu Qur'an,sebagai member platinum shopee kecewa rasanya ...,1,2025-04-08 13:21:33
3,Aat Atmanah,kadang kesel kalau tokonya yang ngirim sesuai ...,5,2025-04-08 13:19:40
4,Alexander Darren,"belanja enak,,banyak diskon",5,2025-04-08 13:19:36


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  20000 non-null  object
 1   content   20000 non-null  object
 2   score     20000 non-null  int64 
 3   at        20000 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.1+ KB


**Insight**
- Tidak ada missing values pada dataset

In [5]:
duplicate_count = df.duplicated(subset=['content']).sum()
print(f'Jumlah duplicate data: {duplicate_count}')

Jumlah duplicate data: 5254


In [6]:
df_clean = df.drop_duplicates(subset=['content']).reset_index(drop=True)

print(f'Jumlah data setelah hapus duplicate: {len(df_clean)}')

Jumlah data setelah hapus duplicate: 14746


**Insigth**
- Sesuai dengan ketentuan minimal sampel data yaitu 10ribu

<h1> Text Pre Processing

In [7]:
import re
import string
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [10]:
#1. Cleaning: Membersihkan teks dari hal-hal tidak perlu
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Menghapus link
    text = re.sub(r"\d+", '', text) # Menghapus angka
    text = re.sub(r"[^\w\s]", '', text) # Menghapus simbol
    text = text.replace('\n', ' ') # Mengganti newline dengan spasi
    text = text.strip()
    return text

#2. Konversi ke huruf kecil
def casefoldingText(text):
    text = text.lower()
    return text

#3. Tokenisasi
def tokenizingText(text):
    tokens = word_tokenize(text)
    return tokens

#4. Menghapus stopwords (Indonesia + Inggris + tambahan kata umum)
def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update([
        'iya','yaa','gak','nya','na','sih','ku',
        'di','ga','ya','gaa','loh','kah','woi','woii','woy',
        'shopee', 'barang', 'aplikasi', 'belanja',
    ])
    return [word for word in text if word not in listStopwords]

#5. Stemming (Bahasa Indonesia)
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

#6. Ubah list token jadi kalimat lagi
def toSentence(list_words):
    return ' '.join(list_words)

In [11]:
slangwords = {
    "bgt": "banget", "maks": "maksimal", "udh": "sudah", "aja": "saja", "blm": "belum", 
    "trs": "terus", "tdk": "tidak", "gk": "tidak", "ga": "tidak", "dr": "dari", "dpt": "dapat",
    "brg": "barang", "bgus": "bagus", "gpp": "tidak apa", "sgt": "sangat"
}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text